In [1]:
import numpy as np
import cvxpy as cp
import random
import pandas as pd
import math

In [2]:
n = 10000    # number of bidders
m = 10       # type of resources

A = pd.read_csv("A.csv")
pi = pd.read_csv("pi_2.csv")
pi = np.array(pi)
pi = pi.T
A = np.array(A)
b = np.array([1000 for _ in range(m)])  



In [3]:
x = cp.Variable(n)
ee = np.ones(n)
objective = cp.Maximize(pi@x)
constraints = [A@x <= b, x >= 0, x <= ee]
prob = cp.Problem(objective, constraints)
prob.solve()
opv1 = prob.value
print("offline optimal value, ", opv1)

offline optimal value,  17406.295176501088


In [4]:
l_decision = []
b_k = b
price = [100 for _ in range(m)]
x = 0
flag = 1
AT = A.T
for k in range(n):
    flag = 1
    # Check whether remaining certain resources needed for k
    for i in range(len(b_k)):
        if A[:,k][i] > b_k[i]:
            # print("cannot satisfy")
            l_decision.append(0)
            flag = 0
            break
    if flag == 0:
        continue

    # Decide whether to accept (k)th request and the remaining resource
    if pi[0][k] > AT[k].dot(price):
        x = 1
        b_k = b_k - A[:,k]
    else:
        x = 0
    l_decision.append(x)

    # Update the dual price after making the decision
    A1 = A[:, :k+1]
    pik = pi[:,:k+1]
    print("ak", A[:,k+1])
    print("bk", b_k) 
    Anow = A1.T
    y = cp.Variable((m+k+1))
    e = np.ones(k+1)
    bnow = b_k * (float((k+1)/(n-k+1)))
    pNow = np.reshape(pik, (k+1,)) 
    # print(bnow.shape)
    # print(y[:m].shape)
    # print(e.shape)
    # print(y[m:].shape)
    obj = cp.Minimize(bnow@(y[:m]) + e@(y[m:]))
    cons = [(Anow@(y[:m])+ (y[m:]))>= pNow, y >= 0]
    prob = cp.Problem(obj, cons)
    prob.solve()
    price = y[:m].value
    
    print("k=",k+1,pik@l_decision-(k+1/n)*opv1)
    # print(price)
l_decision
print(len(l_decision))

ak [0 1 0 0 1 0 0 0 1 1]
bk [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
k= 1 [-1.74062952]
ak [0 0 0 0 1 1 1 1 0 1]
bk [1000  999 1000 1000  999 1000 1000 1000  999  999]
k= 2 [-17401.96723015]
ak [0 0 0 0 0 0 0 1 0 1]
bk [1000  999 1000 1000  999 1000 1000 1000  999  999]
k= 3 [-34808.26240665]
ak [1 1 1 0 1 0 1 0 0 0]
bk [1000  999 1000 1000  999 1000 1000  999  999  998]
k= 4 [-52209.60092904]
ak [1 0 0 1 0 1 0 1 1 0]
bk [ 999  998  999 1000  998 1000  999  999  999  998]
k= 5 [-69610.81555287]
ak [1 1 1 0 1 1 0 0 0 1]
bk [998 998 999 999 998 999 999 998 998 998]
k= 6 [-87012.15196146]
ak [0 0 0 0 1 0 0 0 1 1]
bk [998 998 999 999 998 999 999 998 998 998]
k= 7 [-104418.44713796]
ak [0 1 1 0 0 0 0 0 0 0]
bk [998 998 999 999 997 999 999 998 997 997]
k= 8 [-121818.55234588]
ak [0 0 0 1 0 0 0 1 0 0]
bk [998 997 998 999 997 999 999 998 997 997]
k= 9 [-139223.10349439]
ak [1 0 1 0 0 0 1 1 1 1]
bk [998 997 998 998 997 999 999 997 997 997]
k= 10 [-156623.69005297]
ak [0 0 1 0 1 0 0 0

In [5]:
print(len(l_decision))
print(pi.shape)
print(pi@l_decision)
print(pi@l_decision - opv1)

10000
(1, 10000)
[17290.93306312]
[-115.36211338]


In [6]:

opv2 = (pi.dot(l_decision))[0]
print("optimal value:", opv2)
print("competetive ratio", opv2/opv1)
        

optimal value: 17290.933063118122
competetive ratio 0.9933723912979077


In [7]:
x = cp.Variable(n)
ee = np.ones(n)
objective = cp.Maximize(pi@x)
constraints = [A@x <= b, x >= 0, x <= ee]
prob = cp.Problem(objective, constraints)

prob.solve()
x.value
opv1 = prob.value
print("competetive ratio", opv2/opv1)

competetive ratio 0.9933723912979077
